In [39]:
## 分析筛选GWAS 位点
import pandas as pd
import numpy as np
## 筛选过阈值点的, 并且确定对应区间
def top_point_region(df, size):
    df_true_point = df.query("p_score<=0.000001")[['chr', 'ps']].copy()
    df_true_point['start'] = df_true_point['ps'] - size
    df_true_point['end'] = df_true_point['ps'] + size
    #print(df_true_point)
    return df_true_point
## 确定区间范围所包含的snp 位点
def region_snp(df, c, s, e):
    #print(c, s, e)
    df = df.query("chr==@c & ps>= @s & ps<=@e").copy()
    #print(df)
    return df
## 判断位点是否连续
def if_continus(df):
    df['log10p'] = df['p_score'].apply(np.log10)
    df['log10p_negative'] = -df['log10p']
    #print(df.head())
    n = 0 
    for i in np.arange(0, 6, 0.6):
        a, b = i, i+0.5
        df_tmp = df.query("log10p_negative>=@a & log10p_negative<=@b").copy()
        if df_tmp.shape[0]>0:
            n += 1
    #print(n)
    if n >= 8:
        df['tags'] = ['continue']*df.shape[0]
    else:
        df['tags'] = ['No_continue']*df.shape[0]
    return df

## 根据对应区间筛选对应snp位点
def all_region_snp(df_ps, df_snp):
    df_return = pd.DataFrame()
    k = 0
    for index, row in df_ps.iterrows():
        ps_n = "ps_{}".format(k)
        c, s, e = row['chr'], row['start'], row['end']
        df_tmp = region_snp(df_snp, c, s, e) # 选择区间
        df_tmp = if_continus(df_tmp) # 确定区间是否连续，打上标签
        df_tmp['ps_n'] = [ps_n]*df_tmp.shape[0] # 打上位置标签
        df_return = pd.concat([df_return, df_tmp], axis=0) # 
        k = k+1
        #print(df_tmp)
    return df_return

# ## 分析确定有哪些snp位点是可用的, D3
df_result = pd.DataFrame()
for i in range(1, 102):
    input_f = "G:/Billfish/VS_code_program/python/yqming/test_data/D3_output/gemma_{}.assoc.txt".format(i)
    df = pd.read_csv(input_f, sep="\t")
    df_top_snp = top_point_region(df, 200000) ## 确定顶点区域
    df_choose_snp_region = all_region_snp(df_top_snp, df) # 选择连续的区间
    df_choose_snp_region['t_tags'] = ["gemma_{}".format(i)]*df_choose_snp_region.shape[0]
    df_result = pd.concat([df_result, df_choose_snp_region], axis=0)
df_result.to_csv("G:/Billfish/VS_code_program/python/yqming/test_data/D3_output/D3_SNP_result.bed", sep="\t", index=False)

## 分析确定有哪些snp位点是可用的, D12
# df_result = pd.DataFrame()
# for i in range(1, 102):
#     input_f = "G:/Billfish/VS_code_program/python/yqming/test_data/D12_output/gemma_{}.assoc.txt".format(i)
#     df = pd.read_csv(input_f, sep="\t")
#     df_top_snp = top_point_region(df, 200000) ## 确定顶点区域
#     df_choose_snp_region = all_region_snp(df_top_snp, df) # 选择连续的区间
#     df_choose_snp_region['t_tags'] = ["gemma_{}".format(i)]*df_choose_snp_region.shape[0]
#     df_result = pd.concat([df_result, df_choose_snp_region], axis=0)
# df_result.to_csv("G:/Billfish/VS_code_program/python/yqming/test_data/D12_output/D12_SNP_result.bed", sep="\t", index=False)


In [41]:
df = pd.read_csv("G:/Billfish/VS_code_program/python/yqming/test_data/D12_output/D12_SNP_result.bed", sep="\t")
df_choose = df.query("tags=='continue'").copy()
df_choose.to_csv("G:/Billfish/VS_code_program/python/yqming/test_data/D12_output/D12_SNP_result_continue.bed", sep="\t")
df_choose['number'] = [1]*df_choose.shape[0]
df_choose.drop_duplicates(keep='first', subset=['ps_n', 'tags'], inplace=True)
df_sta = df_choose.groupby(by=['ps_n', 'tags'], as_index=False)['number'].sum()
print(df_sta)

       ps_n      tags  number
0      ps_0  continue       1
1      ps_1  continue       1
2     ps_10  continue       1
3    ps_100  continue       1
4    ps_101  continue       1
..      ...       ...     ...
843   ps_95  continue       1
844   ps_96  continue       1
845   ps_97  continue       1
846   ps_98  continue       1
847   ps_99  continue       1

[848 rows x 3 columns]


In [43]:
## 去除重复信息
import pandas as pd
df = pd.read_csv("G:/Billfish/VS_code_program/python/yqming/test_data/D3_output/D3_SNP_result_continue.bed", sep="\t")
df.drop_duplicates(keep='first', subset=['chr','rs', 'ps'], inplace=True)
df.to_csv("G:/Billfish/VS_code_program/python/yqming/test_data/D3_output/D3_SNP_result_continue_norepeat.bed", sep="\t", index=False)